# Hit graph construction

This is a notebook for developing and analyzing the procedure for constructing hit graphs.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from xtracker.graph_creation import (
    calc_dphi, calc_eta, construct_graph, select_hits, split_detector_sections, form_layer_pairs, 
    construct_segments
)


## Read the event data

In [ ]:
input_dir = "/home/benjamin/xtracker/examples/data/events_belle2_vtx"

In [ ]:
evtid = 300

hits = pd.read_hdf(os.path.expandvars( input_dir + '/event_id_{}.h5'.format(evtid+1) ), 'hits')
truth = pd.read_hdf(os.path.expandvars( input_dir + '/event_id_{}.h5'.format(evtid+1) ), 'truth')
particles = pd.read_hdf(os.path.expandvars( input_dir + '/event_id_{}.h5'.format(evtid+1) ), 'particles')
trigger = pd.read_hdf(os.path.expandvars( input_dir + '/event_id_{}.h5'.format(evtid+1) ), 'trigger')

In [ ]:
hits.tail()

In [ ]:
particles.tail()

In [ ]:
truth.tail()

In [ ]:
trigger.tail()

## Hit selection

In [ ]:
pt_min = 0. # GeV

In [ ]:
hits = (select_hits(hits, truth, particles, pt_min=pt_min)
        .assign(evtid=0)
        .reset_index(drop=True))

## Geometry ID pairs

We use geometry IDs to select initial set of hit pair segments.
For now we're starting with barrel hits only and can use the layer number as the ID.
We'll then use consecutive layer numbers as the criteria.

In [ ]:
n_det_layers = 5
segment_type = 'all'


layer_pairs = form_layer_pairs(n_det_layers, segment_type)

## Segment construction

Now for every layer pair we construct hit-pair segments.

In [ ]:
def get_segments(hits, layer_pairs):

    # Group hits by geometry ID
    layer_groups = hits.groupby('layer')

    segments = []
    
    for (layer1, layer2) in layer_pairs:
        # Find and join all hit pairs
        try:
            hits1 = layer_groups.get_group(layer1)
            hits2 = layer_groups.get_group(layer2)
        # If an event has no hits on a layer, we get a KeyError.
        # In that case we just skip to the next layer pair
        except KeyError as e:
            continue
        # Construct the segments
        
        # Start with all possible pairs of hits
        keys = ['evtid', 'r', 'phi', 'z', 'particle_id', 'hit_id', 'layer']
        hit_pairs = hits1[keys].reset_index().merge(
            hits2[keys].reset_index(), on='evtid', suffixes=('_1', '_2'))
        
        # Compute line through the points
        dphi = calc_dphi(hit_pairs.phi_1, hit_pairs.phi_2)
        dz = hit_pairs.z_2 - hit_pairs.z_1
        dr = hit_pairs.r_2 - hit_pairs.r_1
        phi_slope = dphi / dr
        z0 = hit_pairs.z_1 - hit_pairs.r_1 * dz / dr
        
        # We do not have good filter for same layer or ingoing edges
        z0[ hit_pairs.layer_1 >= hit_pairs.layer_2 ] = 0.0 
        phi_slope[ hit_pairs.layer_1 == hit_pairs.layer_2 ] = 0.0 
        
        # Identify the true pairs
        y = (hit_pairs.particle_id_1 == hit_pairs.particle_id_2) & (hit_pairs.hit_id_1+1 == hit_pairs.hit_id_2)
            
        # Put the results in a new dataframe
        segments.append(hit_pairs[['evtid', 'index_1', 'index_2', 'layer_1', 'layer_2']]
                        .assign(dphi=dphi, dz=dz, dr=dr, y=y, phi_slope=phi_slope, z0=z0))
        
    return pd.concat(segments, ignore_index=True)

In [ ]:
segments = get_segments(hits, layer_pairs)

In [ ]:
segments.describe()

## Plot the full segment distributions

In [ ]:
plt.figure(figsize=(14,6))

true_segs = segments[segments.y]
fake_segs = segments[segments.y == False]

plt.subplot(121)
binning=dict(bins=150, range=(-2.2, 2.2))
plt.hist(fake_segs.phi_slope, label='fake', log=True, **binning)
plt.hist(true_segs.phi_slope, label='true', **binning)
plt.xlabel('$\Delta \phi / \Delta r$ [rad/mm]')
plt.legend(loc=0)

plt.subplot(122)
binning=dict(bins=50, range=(-50, 50))
plt.hist(fake_segs.z0, label='fake', log=True, **binning)
plt.hist(true_segs.z0, label='true', **binning)
plt.xlabel('$z_0$ [mm]')
plt.legend(loc=0)

plt.tight_layout()

## Segment selection

In [ ]:
def select_segments(segments, phi_slope_min, phi_slope_max, z0_max):
    sel_mask = ((segments.phi_slope > phi_slope_min) &
                (segments.phi_slope < phi_slope_max) &
                (segments.z0 < z0_max) &  (segments.z0 > -z0_max) )
    return segments.assign(selected=sel_mask)

def segment_efficiency(segments):
    return (segments.y & segments.selected).sum() / segments.y.sum()

def segment_purity(segment):
    return (segments.y & segments.selected).sum() / segments.selected.sum()

In [ ]:
# Choose some cuts
phi_slope_min = -2.25
phi_slope_max =  2.25
z0_max =  100

In [ ]:
segments = select_segments(segments, phi_slope_min=phi_slope_min,
                           phi_slope_max=phi_slope_max, z0_max=z0_max)

print('Selection efficiency %.4f purity %.4f' % (segment_efficiency(segments), segment_purity(segments)))

In [ ]:
eff = []
pur = []

for evtid in range(10):
    
    hits = pd.read_hdf(os.path.expandvars( input_dir + '/event_id_{}.h5'.format(evtid+1) ), 'hits')
    truth = pd.read_hdf(os.path.expandvars( input_dir + '/event_id_{}.h5'.format(evtid+1) ), 'truth')
    particles = pd.read_hdf(os.path.expandvars( input_dir + '/event_id_{}.h5'.format(evtid+1) ), 'particles')
    
    
    hits = (select_hits(hits, truth, particles, pt_min=pt_min)
        .assign(evtid=0)
        .reset_index(drop=True))
    
    layer_pairs = form_layer_pairs(n_det_layers, segment_type)
    
    segments = get_segments(hits, layer_pairs)
    
    segments = select_segments(segments, phi_slope_min=phi_slope_min,
                           phi_slope_max=phi_slope_max, z0_max=z0_max)
    
    true_segs = segments[segments.y]
    
    eff.append(segment_efficiency(segments))
    pur.append(segment_purity(segments))

print('Av. selection efficiency %.4f purity %.4f' % (np.mean(eff), (np.mean(pur))))


    
plt.figure(figsize=(14,6))

plt.subplot(121)
binning=dict(bins=50, range=(0.0, 1.0))
plt.hist(eff, **binning)
plt.xlabel('efficiency')
plt.legend(loc=0)

plt.subplot(122)
binning=dict(bins=50, range=(0, 1.0))
plt.hist(pur, **binning)
plt.xlabel('purity')
plt.legend(loc=0)

plt.tight_layout()    

